In [1]:
from sympy import symbols, simplify, hessian, solveset, S, solve, log, And, Le, Ge, Eq, Lt, Gt

In [2]:
s,
  v,
  w,
  j,
  Delta,
  a,
  min = symbols("s v w j Delta a min", positive = True, integer = True);
b = symbols("b", nonnegative = True, integer = True);
e = symbols("e", integer = True);

## Target function

we want to minimize the effective price, given a set of exponents for the buying
and selling asset each.

In [3]:
deltaBySpot = simplify((s - (v + b) * w) / w);
deltaBySpot;

-b + s/w - v

In [4]:
spotByExp = simplify(a * ((1 + 1 / j) ** e));
spotByExp;

a*((j + 1)/j)**e

In [5]:
deltaByExp = simplify(deltaBySpot.subs(s, spotByExp));
deltaByExp;

a*j**(-e)*(j + 1)**e/w - b - v

### convexity of the target function

- deltaBySpot is linear
- spotByExp is convex
- deltaByExp is a composition of the two

=> deltaByExp is convex

- negation flips convexity/concavity
- deltaBuyingByExp is negative => -deltaBuyingByExp is positive
- inversion of a positive function flips convexity/concavity => 1 /
  (-deltaBuyingByExp) is convex
- multiplying two nonnegative functions with different variables, that are each
  convex and independent of the other's variable, results in a convex function

==> effectivePrice = deltaSellingByExp / (-deltaBuyingByExp) is convex. qed

## inequality-constraints

- the exponents must adhere to their upper (buying) resp. lower (selling) bounds
  given by our equation
- need to buy and sell minimum amounts (TODO)
- cannot buy more than the available balance
- cannot sell more than maxSelling
- the spot prices must not exceed maxInteger
- bonus: the total number of multiplications for both exponentiations must not
  exceed expLimit (TODO)

In [6]:
expBound = simplify(log(w * (v + b) / a, 1 + 1 / j));
expBound;

log((w*(b + v)/a)**(1/log((j + 1)/j)))

In [7]:
expBoundBuying = e - expBound # <=! 0 (upper bound)
expBoundBuying

e - log((w*(b + v)/a)**(1/log((j + 1)/j)))

In [8]:
expBoundSelling = expBound - e # <=! 0 (lower bound)
expBoundSelling

-e + log((w*(b + v)/a)**(1/log((j + 1)/j)))

In [9]:
maxBuyingBound = -deltaByExp - b # <=! 0
maxBuyingBound

-a*j**(-e)*(j + 1)**e/w + v

In [10]:
max_s, max_I = symbols('max_s, max_I', positive=True, integer=True)

maxSellingBound = deltaByExp - max_s # <=! 0
maxSellingBound

a*j**(-e)*(j + 1)**e/w - b - max_s - v

In [11]:
maxSpotBound = spotByExp - max_I # <=! 0
maxSpotBound

a*((j + 1)/j)**e - max_I

## Lagrangian

In [12]:
l1, l2, l3, l4, l5, l6 = symbols('lambda_1 lambda_2 lambda_3 lambda_4 lambda_5 lambda_6', positive=True)

# we are trying to maximize Delta in both cases, considering that buying-Delta is negative.
L_b = -deltaByExp + l1 * expBoundBuying# + l2 * maxBuyingBound + l3 * maxSpotBound
L_b

-a*j**(-e)*(j + 1)**e/w + b + lambda_1*(e - log((w*(b + v)/a)**(1/log((j + 1)/j)))) + v

In [13]:
# we are trying to maximize Delta in both cases, considering that buying-Delta is negative.
L_s = -deltaByExp + l1 * expBoundSelling# + l2 * maxSellingBound + l3 * maxSpotBound
L_s

-a*j**(-e)*(j + 1)**e/w + b + lambda_1*(-e + log((w*(b + v)/a)**(1/log((j + 1)/j)))) + v

In [14]:
# Partial derivatives
dL_b_de = L_b.diff(e)
dL_b_dl1 = L_b.diff(l1)
# dL_dl2 = L.diff(l2)
# dL_dl3 = L.diff(l3)
# dL_dl4 = L.diff(l4)
# dL_dl5 = L.diff(l5)
# dL_dl6 = L.diff(l6)

# Set them to zero
stationary_conditions_b = [
    Eq(dL_b_de, 0),
    Eq(dL_b_dl1, 0),
    # Eq(dL_dl2, 0),
    # Eq(dL_dl3, 0),
    # Eq(dL_dl4, 0),
    # Eq(dL_dl5, 0),
    # Eq(dL_dl6, 0),
]

In [15]:
# Partial derivatives
dL_s_de = L_s.diff(e)
dL_s_dl1 = L_s.diff(l1)
# dL_dl2 = L.diff(l2)
# dL_dl3 = L.diff(l3)
# dL_dl4 = L.diff(l4)
# dL_dl5 = L.diff(l5)
# dL_dl6 = L.diff(l6)

# Set them to zero
stationary_conditions_s = [
    Eq(dL_s_de, 0),
    Eq(dL_s_dl1, 0),
    # Eq(dL_dl2, 0),
    # Eq(dL_dl3, 0),
    # Eq(dL_dl4, 0),
    # Eq(dL_dl5, 0),
    # Eq(dL_dl6, 0),
]

In [16]:
solutions_b = solve(stationary_conditions_b, (e, l1))#, l2, l3))#, l5, l6))

solutions_b = solutions_b[0]
# e_s_sol, e_b_sol, l1_sol, l2_sol, l3_sol, l4_sol, l5_sol, l6_sol = solutions
e_b_sol, l1_b_sol = solutions_b
solutions_b

(log((a/(w*(b + v)))**(1/(log(j) - log(j + 1)))),
 log(((j + 1)/j)**(a*j**(-log(a**(1/(log(j) - log(j + 1)))*(w*(b + v))**(-1/(log(j) - log(j + 1)))))*(j + 1)**log(a**(1/(log(j) - log(j + 1)))*(w*(b + v))**(-1/(log(j) - log(j + 1))))/w)))

In [17]:
solutions_s = solve(stationary_conditions_s, (e, l1))#, l2, l3))#, l5, l6))

solutions_s = solutions_s[0]
# e_s_sol, e_b_sol, l1_sol, l2_sol, l3_sol, l4_sol, l5_sol, l6_sol = solutions
e_s_sol, l1_s_sol = solutions_s
solutions_s

(log((a/(w*(b + v)))**(1/(log(j) - log(j + 1)))),
 log((j/(j + 1))**(a*j**(-log(a**(1/(log(j) - log(j + 1)))*(w*(b + v))**(-1/(log(j) - log(j + 1)))))*(j + 1)**log(a**(1/(log(j) - log(j + 1)))*(w*(b + v))**(-1/(log(j) - log(j + 1))))/w)))

In [18]:
e_b_sol;

log((a/(w*(b + v)))**(1/(log(j) - log(j + 1))))

In [19]:
e_s_sol;

log((a/(w*(b + v)))**(1/(log(j) - log(j + 1))))

In [20]:
l1_b_sol;

log(((j + 1)/j)**(a*j**(-log(a**(1/(log(j) - log(j + 1)))*(w*(b + v))**(-1/(log(j) - log(j + 1)))))*(j + 1)**log(a**(1/(log(j) - log(j + 1)))*(w*(b + v))**(-1/(log(j) - log(j + 1))))/w))

In [21]:
l1_s_sol;

log((j/(j + 1))**(a*j**(-log(a**(1/(log(j) - log(j + 1)))*(w*(b + v))**(-1/(log(j) - log(j + 1)))))*(j + 1)**log(a**(1/(log(j) - log(j + 1)))*(w*(b + v))**(-1/(log(j) - log(j + 1))))/w))

In [22]:
for sol in solutions_b:
    print(sol.free_symbols)
for sol in solutions_s:
    print(sol.free_symbols)

{j, w, v, b, a}
{j, w, v, b, a}
{j, w, v, b, a}
{j, w, v, b, a}


In [23]:
simplify(e_b_sol - e_s_sol);

0

In [24]:
simplify(l1_b_sol + l1_s_sol);

0

In [25]:
print(l1_b_sol);

log(((j + 1)/j)**(a*j**(-log(a**(1/(log(j) - log(j + 1)))*(w*(b + v))**(-1/(log(j) - log(j + 1)))))*(j + 1)**log(a**(1/(log(j) - log(j + 1)))*(w*(b + v))**(-1/(log(j) - log(j + 1))))/w))


In [26]:
u1, u2 = symbols("u1 u2");

l1_sol = l1_b_sol.subs(1 / (log(j) - log(j + 1)), u1);
l1_sol = l1_sol.subs(log(a ** u1 * (w * (b + v)) ** (-u1)), u2);
l1_sol;

log(((j + 1)/j)**(a*j**(-u2)*(j + 1)**u2/w))

l1_sol must be nonnegative -> logarithm is nonnegative when argument is >= 1

In [27]:
l1_sol.args[0];

((j + 1)/j)**(a*j**(-u2)*(j + 1)**u2/w)

base is < 1